In [1]:
import single_head
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
import pickle
np.set_printoptions(threshold=sys.maxsize)

%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
# load heads
all_heads=[single_head.SingleHead.load_from_pickle(1,i) for i in range (1,16)]

In [6]:
# make head from first to last
def make_mhead(all_heads, first, last):
    all_heads[first].reset_positions()
    all_heads[first].reset_colors()
    all_heads[first+1].reset_positions()
    all_heads[first+1].reset_colors()   
    heads = multi_head.MultiHead.joined_heads(all_heads[first], all_heads[first+1])
    heads.links.append(multi_head.Link(left=all_heads[first+1].frame_id,right=all_heads[first].frame_id))
    for i in range(first+2,last+1):
        heads.links.append(multi_head.Link(left=all_heads[i].frame_id,right=all_heads[i-1].frame_id))
        all_heads[i].reset_positions()
        all_heads[i].reset_colors()
        heads.append(all_heads[i])        
        if i==last:
             heads.links.append(multi_head.Link(left=all_heads[first].frame_id,right=all_heads[i].frame_id))
    print ("completed")
    return heads

mhead = make_mhead(all_heads, 0, 14)
mhead.calc_keypoints()

completed


In [7]:
def calc_connections(mhead):
    for link in mhead.links:
        link.reset()
        mhead.ransac_from_link(link)        
calc_connections(mhead)

/Users/caseliang/Repositories/AV_CW/SIFT.py:84: RuntimeWarning: divide by zero encountered in double_scalars
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))
/Users/caseliang/Repositories/AV_CW/SIFT.py:84: RuntimeWarning: invalid value encountered in sqrt
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))


In [8]:
link_index,err=mhead.get_next_unpositioned_link()
while(link_index):
    mhead.links[link_index].print_short()
    foo= mhead.transform_from_link(mhead.links[link_index])
    link_index,err=mhead.get_next_unpositioned_link()

any_head_positioned False
6-5, Count=109, Err=0.0033
any_head_positioned True
5-4, Count=140, Err=0.0039
any_head_positioned True
4-3, Count=122, Err=0.0039
any_head_positioned True
3-2, Count=152, Err=0.0051
any_head_positioned True
